In [ ]:
"""

Title: "Neural Barrier Certificates for Monotone Systems"
Code for: Learing a mixed-monotone neural barrier certificate for a 2-D linear example.
Author: Amirreza Alavi
Contact: seyedamirreza.alavi@colorado.edu
Affiliation: University of Colorado Boulder
Date: 03/2025

"""

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy import stats
from functools import partial
%matplotlib inline
import numpy as np
from tensorflow.keras.layers import Input, Dense, Concatenate, Add
from tensorflow.keras import Model
from tensorflow.keras import regularizers
import time 
import matplotlib.patches as mpatches

In [ ]:
def cart(x,y):
    temp = np.zeros([x.shape[0]*y.shape[0],x.shape[1]+y.shape[1]])
    j=0
    num2 = x.shape[0]
    num1 = x.shape[1]
    for i in range(temp.shape[0]):
        if i!=0 and i%num2 == 0:
            j+=1
        temp[i,0:num1] = x[i%num2]
        temp[i,num1::] = y[j]
    return temp

In [ ]:
z0bar = np.array([[-1 , 2]])
z0und = np.array([[-2 , 1]])

smpnz10=5
smpnz20=5
z10 = np.linspace(-2,-1,smpnz10).reshape(smpnz10,1)
z20 = np.linspace(1,2,smpnz20).reshape(smpnz20,1)
z0 = cart (z10,z20)

zubar = np.array([[2 ,  1],
                  [1 ,  -1]])
zuund = np.array([[1 ,  -2],
                  [0 ,  -2]])

smpnz11u=5
smpnz21u=13
smpnz12u=5
smpnz22u=5
z11u = np.linspace(1,2,smpnz11u).reshape(smpnz11u,1)
z21u = np.linspace(-2,1,smpnz21u).reshape(smpnz21u,1)
zu1 = cart (z11u,z21u)
z12u = np.linspace(0,1,smpnz12u).reshape(smpnz12u,1)
z22u = np.linspace(-2,-1,smpnz22u).reshape(smpnz22u,1)
zu2 = cart (z12u,z22u)
zu = np.concatenate((zu1, zu2), axis=0)

smpnz1=5
smpnz2=5
z1 = np.linspace(-2, 2, smpnz1).reshape(smpnz1,1)
z2 = np.linspace(-2, 2, smpnz2).reshape(smpnz2,1)
z = cart (z1,z2)

pairs = np.zeros((((smpnz1-1)*(smpnz2)),2,2))
excluded_numbers = {i for i in range(smpnz1 - 1, (smpnz1 * smpnz2) - (smpnz1+1) , smpnz1)}
for i in range((smpnz1 * smpnz2) - (smpnz1+1)):
    if i not in excluded_numbers:
        pairs [i,:,:] = np.array([z[i + smpnz1 + 1], z[i]])

zz = pairs[~np.all(pairs == 0, axis=(1, 2))]

unsafe_mask = (
    ((zz[:, 0, 0] >= 1) & (zz[:, 0, 0] <= 2) &   
     (zz[:, 0, 1] >= -2) & (zz[:, 0, 1] <= 1)) &   
    ((zz[:, 1, 0] >= 1) & (zz[:, 1, 0] <= 2) &   
     (zz[:, 1, 1] >= -2) & (zz[:, 1, 1] <= 1))     
)

zz_filtered = zz[~unsafe_mask]

unsafe_mask2 = (
    ((zz_filtered[:, 0, 0] >= 0) & (zz_filtered[:, 0, 0] <= 1) &   
     (zz_filtered[:, 0, 1] >= -2) & (zz_filtered[:, 0, 1] <= -1)) &   
    ((zz_filtered[:, 1, 0] >= 0) & (zz_filtered[:, 1, 0] <= 1) &   
     (zz_filtered[:, 1, 1] >= -2) & (zz_filtered[:, 1, 1] <= -1))     
)

zz_filtered2 = zz_filtered[~unsafe_mask2]


zbar = zz_filtered2 [:,0,:]
zund = zz_filtered2 [:,1,:]

h=0.1

z1bar = tf.reshape(zbar[:,0].astype(np.float32),[zbar.shape[0],1])
z2bar = tf.reshape(zbar[:,1].astype(np.float32),[zbar.shape[0],1])

z1und = tf.reshape(zund[:,0].astype(np.float32),[zund.shape[0],1])
z2und = tf.reshape(zund[:,1].astype(np.float32),[zund.shape[0],1])

z1barn = (1-2*h)*z1bar + (h)*z2bar - h
z2barn = (h)*z1bar + (1-2*h)*z2bar + h

z1undn = (1-2*h)*z1und + (h)*z2und - h
z2undn = (h)*z1und + (1-2*h)*z2und + h

fzbar = tf.reshape(tf.stack([z1barn,z2barn],axis=1),zbar.shape)
fzund = tf.reshape(tf.stack([z1undn,z2undn],axis=1),zund.shape)

In [ ]:
# H1

modelh1 = tf.keras.Sequential()
modelh1.add(layers.Dense(40, use_bias=True, activation = 'tanh' ,input_shape=(2,),kernel_constraint=tf.keras.constraints.NonNeg()))
modelh1.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh1.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh1.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh1.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh1.add(layers.Dense(1,use_bias=True, activation = 'linear',kernel_constraint=tf.keras.constraints.NonNeg()))

# H2

modelh2 = tf.keras.Sequential()
modelh2.add(layers.Dense(40, use_bias=True, activation = 'tanh' , input_shape=(2,),kernel_constraint=tf.keras.constraints.NonNeg()))
modelh2.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh2.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh2.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh2.add(layers.Dense(60, use_bias=True, activation = 'tanh' , kernel_constraint=tf.keras.constraints.NonNeg()))
modelh2.add(layers.Dense(1,use_bias=True, activation = 'linear',kernel_constraint=tf.keras.constraints.NonNeg()))


opt1 = tf.keras.optimizers.Adam(1e-2)
opt2 = tf.keras.optimizers.Adam(1e-2)

In [ ]:
epochs = 300000

t1 = time.perf_counter()
etaa=0.0000001
eta=0.0001

for i in range(epochs):
    
    with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
        
        
        H1_z0bar = tf.reshape(modelh1(z0bar, training=True), [z0bar.shape[0], 1])
        H1_z0und = tf.reshape(modelh1(z0und, training=True), [z0und.shape[0], 1])
        H1_zubar = tf.reshape(modelh1(zubar, training=True), [zubar.shape[0], 1])
        H1_zuund = tf.reshape(modelh1(zuund, training=True), [zuund.shape[0], 1])
        H1_zund = tf.reshape(modelh1(zund, training=True), [zund.shape[0], 1])
        H1_zbar = tf.reshape(modelh1(zbar, training=True), [zbar.shape[0], 1])
        
        H2_z0und = tf.reshape(modelh2(z0und, training=True), [z0und.shape[0], 1])
        H2_z0bar = tf.reshape(modelh2(z0bar, training=True), [z0bar.shape[0], 1])
        H2_zuund = tf.reshape(modelh2(zuund, training=True), [zuund.shape[0], 1])
        H2_zubar = tf.reshape(modelh2(zubar, training=True), [zubar.shape[0], 1])
        H2_zund = tf.reshape(modelh2(zund, training=True), [zund.shape[0], 1])
        H2_zbar = tf.reshape(modelh2(zbar, training=True), [zbar.shape[0], 1])

        H1fzbar = tf.reshape(modelh1(fzbar, training=True), [fzbar.shape[0], 1])
        H2fzund = tf.reshape(modelh2(fzund, training=True), [fzund.shape[0], 1])

   
        loss_h3 = tf.reduce_sum(tf.nn.relu(tf.subtract(eta,eta)))
        loss_h4 = tf.reduce_sum(tf.nn.relu(tf.subtract(eta,eta)))


        loss_h3 += tf.reduce_sum(tf.nn.relu(tf.subtract(H1_z0bar-H2_z0und,-etaa))) 
        loss_h3 += tf.reduce_sum(tf.nn.relu(0-(H1_zuund-H2_zubar)))
        loss_h3 += tf.reduce_sum(tf.math.sign(tf.nn.relu(-(H1_zund-H2_zbar)))* tf.nn.relu(H1fzbar-H2fzund))

        
        loss_h4 += tf.reduce_sum(tf.square(tf.nn.relu(H1_z0bar-H2_z0und + 15))) 
        loss_h4 += tf.reduce_sum(tf.square(tf.nn.relu(etaa-H1_zuund+H2_zubar)))
        loss_h4 += tf.reduce_sum(tf.math.sign(tf.nn.relu(-(H1_zund-H2_zbar)))* tf.square(tf.nn.relu(H1fzbar-H2fzund)))


        if tf.reduce_sum(loss_h3) == 0 and i > 11:
            print(f"Training terminated at epoch {i} as loss reached exactly zero.")
            break
        
        grad_h1 = tape0.gradient(loss_h4, modelh1.trainable_variables)
        grad_h2 = tape1.gradient(loss_h4, modelh2.trainable_variables)
        
        
        if (i // 10) % 2 == 0:
            clipgrad_h1 = [tf.clip_by_norm(g, 2) for g in grad_h1]
            opt1.apply_gradients(zip(clipgrad_h1, modelh1.trainable_variables))
        else:
            if grad_h2 is not None:
                clipgrad_h2 = [tf.clip_by_norm(g, 2) for g in grad_h2]
                opt2.apply_gradients(zip(clipgrad_h2, modelh2.trainable_variables))
        

        if i % 20 == 0:
            print(f"Epoch {i}: loss_h3 = {loss_h3.numpy()}, loss_h4 = {loss_h4.numpy()}")
            
    
t2 = time.perf_counter()
tot = t2 - t1
print(f'Total runtime: {tot}s')


In [ ]:
x = np.linspace(-2, 2, 500)  
y = np.linspace(-2, 2, 500)
z1, z2 = np.meshgrid(x, y)
z = np.stack([z1, z2], axis=-1).reshape(-1, 2) 

modelh1_output = modelh1.predict(z)  
modelh2_output = modelh2.predict(z)  

condition = modelh1_output - modelh2_output
condition = condition.reshape(500, 500) 

plt.figure(figsize=(5, 4))
plt.scatter(z0[:, 0], z0[:, 1], color='blue', label='Initial Set ($X_0$)')
plt.scatter(zu[:, 0], zu[:, 1], color='red', label='Unsafe Set ($X_u$)')
plt.contourf(z1, z2, condition, levels=[-np.inf, 0], colors=['lightgray'], alpha=0.5)
plt.contour(z1, z2, condition, levels=[0], colors=['black'], alpha=1)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


for i in range(zbar.shape[0]):
    width = zbar[i, 0] - zund[i, 0]
    height = zbar[i, 1] - zund[i, 1]
    rect = plt.Rectangle(zund[i], width, height, linewidth=0.5, edgecolor='green', facecolor='none')
    plt.gca().add_patch(rect)


gray_patch = mpatches.Patch(color='lightgray', label=r'N(x) $\leq$ 0')
plt.legend(handles=[gray_patch] + plt.gca().get_legend_handles_labels()[0], loc='lower left')


plt.rcParams.update({
    "font.weight": "normal",         
    "axes.labelweight": "normal",    
    "axes.titleweight": "normal",    
    "axes.labelsize": 12,          
    "xtick.labelsize": 12,       
    "ytick.labelsize": 12         
})

plt.rcParams["legend.fontsize"] = 11
plt.show()